In [ ]:
import os.path

import pygetm.pyfabm
import pygetm.legacy

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Load North Sea domain
getm_cases = next(filter(os.path.isdir, ('/data/kb/getm-setups', '../../../getm-setups')))
domain = pygetm.legacy.domain_from_topo(os.path.join(getm_cases, 'NorthSea/Topo/NS6nm.v01.nc'), nlev=30, z0_const=0.001)
domain.initialize(2)

In [ ]:
# Load FABM test case jrc-med_ergom and set constant values for all dependencies
m = pygetm.pyfabm.Model('../../extern/fabm/testcases/fabm-jrc-med_ergom.yaml', shape=(domain.nz, domain.ny, domain.nx), libname='fabm_c')
m.setCellThickness(1)
m.interior_dependencies['downwelling_photosynthetic_radiative_flux'].value = 5
m.interior_dependencies['temperature'].value = 15
m.interior_dependencies['practical_salinity'].value = 30
m.horizontal_dependencies['surface_downwelling_photosynthetic_radiative_flux'].value = 50
m.horizontal_dependencies['wind_speed'].value = 50
m.horizontal_dependencies['bottom_stress'].value = 0
m.mask[...] = domain.T.mask
assert m.start()

In [ ]:
# Simulate for 1 day
dt = 1800
for _ in range(int(86400 / dt)):
    sources_int, sources_sf, sources_bt = m.get_sources()
    #print(sources_int.min(), sources_int.max())
    m.interior_state += sources_int * dt
    m.surface_state += sources_sf * dt
    m.bottom_state += sources_bt * dt

In [ ]:
# Plot surface field and y-z slice 
fig, (ax1, ax2) = matplotlib.pyplot.subplots(ncols=2, figsize=(12, 4))
pc = ax1.pcolormesh(m.interior_state[8, -1, ...])
pc = ax2.pcolormesh(m.interior_state[8, :, :, 55])
fig.colorbar(pc);